# Import Libraries

In [1]:
import pandas as pd
import pickle as pkl
import time
import warnings
import string
warnings.simplefilter(action='ignore', category=FutureWarning)

# Read file

In [2]:
books_og = pd.read_csv("data/items_books_latin_some_isbn.csv", low_memory = False, index_col=0).fillna("")

In [4]:
books_og.head()

,ISBN,title,author,year,publisher,alt_title,alt_author
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,Classical Mythology,Mark P. O. Morford|Robert J. Lenardon
1,0801319536,Classical Mythology,Mark P. O. Morford,1998,John Wiley &amp; Sons,Classical Mythology,Mark P. O. Morford|Robert J. Lenardon
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,Clara Callan,Richard Bruce Wright
3,0771597185,The teacher's daughter,Richard Bruce Wright,1982,Macmillan of Canada,The Teacher's Daughter,Richard Bruce Wright
4,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,Decision in Normandy,Carlo D'Este


In [5]:
books = books_og.copy() # make a copy to do analysis

In [101]:
no_results_authors = len(books[(books.alt_author=="")].drop_duplicates(subset = "author"))
no_results_books = len(books[(books.alt_author=="")])

In [46]:
print("For", no_results_authors,"unique authors, we couldn't match the ISBN of at least one of their books.")

For 6463 unique authors, we couldn't match the ISBN of at least one of their books.


In [102]:
print("For", no_results_books,"books, we couldn't match the ISBN.")

For 10097 books, we couldn't match the ISBN.


In [103]:
len(books.drop_duplicates(subset = "author"))

100642

# Check number of autors

Some books have many authors because they are usually essays or short stories.

In [68]:
more_than_one_author = 0
for index, row in books.iterrows():
    alt_author = row.alt_author
    nr_authors = alt_author.count("|") +1
    if nr_authors>1:
        more_than_one_author+=1

In [69]:
print(more_than_one_author)

38068


# Process

In [50]:
first_estimation = len(books[books.author!=books.alt_author])

In [54]:
print("At first glance, we didn't find the same author for", first_estimation,"books out of ~269K. However, it needs more processing.")

At first glance, we didn't find the same author for 90021 books out of ~269K. However, it needs more processing.


## Function to simplify
We need a function that:
1. Removes spaces
2. Removes punctuation
3. Makes lowercase

That way, we can make sure that two strings are the same.

In [60]:
def simplify(name):
    name = name.replace(" ","").translate(str.maketrans('', '', string.punctuation)).lower()
    return name

In [61]:
simplify("Savvina G. Daniil")

'savvinagdaniil'

## Define correct author
First, if author and alt_author are the same, we say that this is the correct author.

In [62]:
books["correct_author"] = ''
for index,row in books.iterrows():
    if simplify(row.author)==simplify(row.alt_author):
        books.at[index, "correct_author"] = row.alt_author

In [66]:
second_estimate = len(books[books.correct_author==""])

In [67]:
print("With the simplifying, we didn't find the same author for", second_estimate,"books out of ~269K. However, it needs even more processing.")

With the simplifying, we didn't find the same author for 79691 books out of ~269K. However, it needs even more processing.


## Collect bag of authors
Since in alt_author we collect all authors, we make sure that we consider if any of them (separated by "|") is the same as author.

In [75]:
for index, row in books.iterrows():
    if row.correct_author=="": # only if we haven't already found the correct author of that book
        alt_author = row.alt_author
        bag_of_alt_authors = alt_author.split("|")
        simplified_bag_of_alt_authors = [simplify(x) for x in bag_of_alt_authors]
        author = simplify(row.author)
        for i in range(len(bag_of_alt_authors)):
            if author==simplified_bag_of_alt_authors[i]:
                books.at[index, "correct_author"] = bag_of_alt_authors[i]

In [77]:
third_estimate = len(books[books.correct_author==""])

In [78]:
print("With accounting for bag of authors, we didn't find the same author for", third_estimate,"books out of ~269K. However, it needs even more processing.")

With accounting for bag of authors, we didn't find the same author for 48711 books out of ~269K. However, it needs even more processing.


I noticed that some names are reversed, e.g. Charles Glass and Glass Charles are not considered the same name by my processing. So I add an extra layer: sort the characters of the name. 

In [79]:
for index, row in books.iterrows():
    if row.correct_author=="": # only if we haven't already found the correct author of that book
        alt_author = row.alt_author
        bag_of_alt_authors = alt_author.split("|")
        simplified_bag_of_alt_authors = [sorted(simplify(x)) for x in bag_of_alt_authors]
        author = sorted(simplify(row.author))
        for i in range(len(bag_of_alt_authors)):
            if author==simplified_bag_of_alt_authors[i]:
                books.at[index, "correct_author"] = bag_of_alt_authors[i]

In [80]:
fourth_estimate = len(books[books.correct_author==""])

In [81]:
print("With the sorting trick, we didn't find the same author for", fourth_estimate,"books out of ~269K. However, it needs even more processing.")

With the sorting trick, we didn't find the same author for 47612 books out of ~269K. However, it needs even more processing.


In [82]:
books[books.correct_author==""]

,ISBN,title,author,year,publisher,alt_title,alt_author,correct_author
8,0688156924,"Clone: The Road to Dolly, and the Path Ahead",Gina Bari Kolata,1998,William Morrow &amp; Company,Clone,Gina Kolata,
22,039592684X,The Best American Short Stories 1999 (Best Ame...,Amy Tan,1999,Mariner Books,The Best American Short Stories 1999,Katrina Kenison,
25,1931056323,The Bonesetter's Daughter,Amy Tan,2001,New Millennium Audio,,,
39,0399141154,Hundred Secret Senses Limited Ed.,Amy Tan,1995,Berkley Publishing Group,,,
40,1590400356,The Kitchen God's Wife,Amy Tan,2002,Phoenix Audio,,,
...,...,...,...,...,...,...,...,...
269246,1893699226,Florida Fog Phantoms (American Chillers),Jonathan Rand,2002,Audio Craft Press,Florida Fog Phantoms,Johnathan Rand,
269254,1555831141,Chrome,George Nadar,1987,Carrier Pigeon,Chrome,George Nader,
269259,0878576568,Biopotency: A Guide in Sexual Success,Ricard Berger,1987,St Martins Pr,BioPotency,Richard E. Berger|Deborah Berger,
269261,0070542023,Basic Scientific Subroutines,Fred Ruckdeschel,1981,Byte Books,BASIC Scientific Subroutines,F. R. Ruckdeschel,


## String Similarity
At this point, we can no longer rely on simply the literal comparison between strings. We need to introduce a similarity measure between them. 

In [87]:
from difflib import SequenceMatcher

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [84]:
similar("Savvina Daniil", "Savvina G. Daniil")

0.9032258064516129

In [85]:
similar("Savvina Daniil", "John Williams")

0.2962962962962963

I will consider every similarity over 0.7 to be the same name.

In [88]:
for index, row in books.iterrows():
    if row.correct_author=="": # only if we haven't already found the correct author of that book
        alt_author = row.alt_author
        bag_of_alt_authors = alt_author.split("|")
        simplified_bag_of_alt_authors = [sorted(simplify(x)) for x in bag_of_alt_authors]
        author = sorted(simplify(row.author))
        
        max_sim = 0
        max_i = -1000
        
        for i in range(len(bag_of_alt_authors)):
            sim = similarity(author, simplified_bag_of_alt_authors[i])
            if sim>max_sim:
                max_sim = sim
                max_i = i
        
        if max_sim>=0.7:
            print(max_sim, bag_of_alt_authors[max_i], row["author"])
            books.at[index, "correct_author"] = bag_of_alt_authors[max_i]

0.8333333333333334 Gina Kolata Gina Bari Kolata
0.926829268292683 Mary-Kate & Ashley Olsen Mary-Kate &amp; Ashley Olsen
0.926829268292683 Mary-Kate & Ashley Olsen Mary-Kate &amp; Ashley Olsen
0.926829268292683 Mary-Kate & Ashley Olsen Mary-Kate &amp; Ashley Olsen
0.926829268292683 Mary-Kate & Ashley Olsen Mary-Kate &amp; Ashley Olsen
0.926829268292683 Mary-Kate & Ashley Olsen Mary-Kate &amp; Ashley Olsen
0.926829268292683 Mary-Kate & Ashley Olsen Mary-Kate &amp; Ashley Olsen
0.926829268292683 Mary-Kate & Ashley Olsen Mary-Kate &amp; Ashley Olsen
0.926829268292683 Mary-Kate & Ashley Olsen Mary-Kate &amp; Ashley Olsen
0.926829268292683 Mary-Kate & Ashley Olsen Mary-Kate &amp; Ashley Olsen
0.926829268292683 Mary-Kate & Ashley Olsen Mary-Kate &amp; Ashley Olsen
0.926829268292683 Mary-kate & Ashley Olsen Mary-Kate &amp; Ashley Olsen
0.926829268292683 Mary-Kate & Ashley Olsen Mary-Kate &amp; Ashley Olsen
0.926829268292683 Mary-Kate & Ashley Olsen Mary-Kate &amp; Ashley Olsen
0.92682926829268

0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein Robert A. Heinlein
0.8823529411764706 Robert Anson Heinlein

0.8571428571428571 Doug Adams Douglas Adams
0.9090909090909091 Paulo Coelho Paolo Coelho
0.9090909090909091 Paulo Coelho Paolo Coelho
0.9696969696969697 Patricia Cornwell Patricia D. Cornwell
0.9696969696969697 Patricia Cornwell Patricia D. Cornwell
0.85 Patricia Daniels Cornwell Patricia D. Cornwell
0.9696969696969697 Patricia Cornwell Patricia D. Cornwell
0.9565217391304348 Carol Farley Carol J. Farley
0.9655172413793104 Elizabeth A. Adler Elizabeth Adler
0.9655172413793104 Elizabeth A. Adler Elizabeth Adler
0.9655172413793104 Elizabeth A. Adler Elizabeth Adler
0.9655172413793104 Elizabeth A. Adler Elizabeth Adler
0.9655172413793104 Elizabeth A. Adler Elizabeth Adler
0.9655172413793104 Elizabeth A. Adler Elizabeth Adler
0.9655172413793104 Elizabeth A. Adler Elizabeth Adler
0.9655172413793104 Elizabeth A. Adler Elizabeth Adler
0.9655172413793104 Elizabeth A. Adler Elizabeth Adler
0.9655172413793104 Elizabeth A. Adler Elizabeth Adler
0.9655172413793104 Elizabeth A. Adler Elizabeth Adle

0.9565217391304348 Wilbur A. Smith Wilbur Smith
0.9565217391304348 Wilbur A. Smith Wilbur Smith
0.9565217391304348 Wilbur A. Smith Wilbur Smith
0.9565217391304348 Wilbur A. Smith Wilbur Smith
0.9473684210526315 Wayne Dyer Wayne W. Dyer
0.7586206896551724 Katherine Applegate K. A. Applegate
0.7586206896551724 Katherine Applegate K. A. Applegate
0.7586206896551724 Katherine Applegate K. A. Applegate
0.7586206896551724 Katherine Applegate K. A. Applegate
0.7586206896551724 Katherine Applegate K. A. Applegate
0.7586206896551724 Katherine Applegate K. A. Applegate
0.7586206896551724 Katherine Applegate K. A. Applegate
0.7586206896551724 Katherine Applegate K. A. Applegate
0.7586206896551724 Katherine Applegate K. A. Applegate
0.7586206896551724 Katherine Applegate K. A. Applegate
0.7586206896551724 Katherine Applegate K. A. Applegate
0.7586206896551724 Katherine Applegate K. A. Applegate
0.7586206896551724 Katherine Applegate K. A. Applegate
0.7586206896551724 Katherine Applegate K. A. Appl

0.967741935483871 Deborah E. McDowell Deborah McDowell
0.9655172413793104 Melisa C. Michaels Melisa Michaels
0.9629629629629629 Vonda N. McIntyre VONDA MCINTYRE
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.9411764705882353 Peter H?eg Peter Hoeg
0.9411764705882353 Peter Heg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 Peter Høeg Peter Hoeg
0.8888888888888888 P

...

0.7619047619047619 Kay Jordan Kohl Kay J. Kohl
0.9411764705882353 Gillian Rubinstein Gillian Rubenstein
0.9411764705882353 C. J. Cherryh C.J. Cherry
0.9375 Joseph A. Califano (Jr.) Joseph A. Califano
0.96 William E. Blank William Blank
0.8666666666666667 Lydia Yuri Minatoya Lydia Minatoya
0.7692307692307693 Cynthia Anne Scott C. Anne Scott
0.8 Honor Arundel H. Arundel
0.8 Jane Asher J. Asher
0.926829268292683 Montreal Health Press The Montreal Health Press
0.95 Nasser Ibrahim Rashid Nasser Ibrahim, Dr. Rashid
0.96 Clarence E. Byrd Clarence Byrd
0.9473684210526315 Alfred Elton Van Vogt A. E. (Alfred Elton) Van Vogt
0.8181818181818182 René J. Garrod Irene Garrod
0.9444444444444444 John Philips Cranwell John Phili Cranwell
0.9090909090909091 Mariano José de Larra Mariano J. De Larra
0.8823529411764706 Felicia Hance Stewart Felicia H. Stewart
0.9090909090909091 Carlota Solé Carlota Sole
0.972972972972973 Lilian Roberts Finlay Lilian Robert Finlay
0.9375 Bernard-Henri Lévy Bernard-Henri Lev

0.896551724137931 Tessa Alb Warschaw Tessa Warschaw
0.896551724137931 Warren Farrell Warren Farrell Ph.D
0.7058823529411765 Angelo Torres L. &amp; Torres, Angelo Siegel
0.7777777777777778 Judith Lannefeld Klinger Judith L. Klinger
0.8275862068965517 Ernest Ludwig Eliel Ernest L. Eliel
0.8 Edward M. Thornton E. M Thornton
0.9655172413793104 Harriet B. Braiker Harriet Braiker
0.9473684210526315 Jane E. Lewis Jane Lewis
0.8275862068965517 Margaret Cooper Gay Margaret C. Gay
0.96 Edward P. Hughes Edward Hughes
0.7692307692307693 Ruth Hatcher O'Mara Ruth H. O'Mara
0.7692307692307693 John Gray J. Gray
0.8717948717948718 Gwendolyn R. Wilson Burton Gwendolyn R. W. Burton
0.9629629629629629 Claude Steiner Claude M. Steiner
0.7878787878787878 Joseph Twadell Shipley Joseph Shipley
0.96 Ivan Moscovich Ian Moscovich
0.7058823529411765 Pierre Amiet P. Amiet
0.9743589743589743 Markella A. Callimassia Markella Callimassia
0.75 Eleanor M. Edwards E.M. Edwards
0.9629629629629629 Timothy Noakes Timothy D

0.782608695652174 Ramsey Campbell R. Campbell
0.8235294117647058 Quobna Ottobah Cugoano Ottobah Cugoano
0.9565217391304348 Anna H. Dickson Anna Dickson
0.8888888888888888 Vicki Ruíz Vicki Ruiz
0.9696969696969697 Annabel M. Patterson Annabel Patterson
0.8421052631578947 Harriette Robinet Harriette Gillem Robinet
0.7391304347826086 They Might Be Giants (Musical group) They Might Be Giants
0.9 Néstor García Canclini Nestor Garcia Canclini
0.7407407407407407 C. W. Gusewelle CHARLES GUSEWELLE
0.9230769230769231 Harald Küppers Harald. Kuppers
0.9696969696969697 Marilynn Sullivan Marilynn C. Sullivan
0.8571428571428571 Jessica E. Hahn Jessica Erica Hahn
0.8780487804878049 Magdalena Ruiz Guiñazú Magdalena Ruiz Guiinazu
0.8888888888888888 David Von Drehle Dave Von Drehle
0.9629629629629629 Viriato Sención Viriato Sencin
0.9565217391304348 Tommy R. Franks Tommy Franks
0.9696969696969697 Stephen E. Whitfield Stephen Whitfield
0.7407407407407407 E. M. Flanagan Jr Flanagan Edward M.
0.8461538461538

0.8888888888888888 John Willis Hurst J. Willis Hurst
0.8235294117647058 Carolyn Buppert Carolyn, Crnp, Jd Buppert
0.9375 Éliane Whitehouse Eliane Whitehouse
0.8148148148148148 Robert Edward Gard Robert E. Gard
0.9285714285714286 Christoph Hein Christopher Hein
0.7 Pierre Gripari Gripari
0.9166666666666666 Byron Preiss Ed Byron Preiss
0.9523809523809523 Carl H. Yaeger Carl Yaeger
0.7058823529411765 Gordon Eklund E. E. and Eklund, Gordon Smith
0.7857142857142857 Betty Weider Joe and Betty Weider
0.9444444444444444 Manuel Álvarez Bravo Manuel Alvarez Bravo
0.7333333333333333 Jetsunma Tenzin Palmo Tenzin Palmo
0.8235294117647058 Max Heindel Heindel
0.96 Philip Caputo Phillip Caputo
0.967741935483871 Stanislas Dehaene Stanisla Dehaene
0.9523809523809523 Sharon J. Kirk Sharon Kirk
0.8085106382978723 Clifford Donald Simak Clifford D. (Clifford Donald) Simak
0.7272727272727273 Robert Escarpit Escarpit
0.8461538461538461 T. T. Henderson Tanya T. Henderson
0.9565217391304348 John Marriott John M

0.9629629629629629 Charles Dewell Charles S. Dewell
0.9696969696969697 Ferdinand G. Mahfood Ferdinand Mahfood
0.96 Shannon Willow Sannon Willow
0.8888888888888888 Beryl T. Sue Atkins Beryl T.  Atkins
0.9166666666666666 David Reuben David, M.D. Reuben
0.9629629629629629 Stan Berenstain Stan Bernstain
0.9032258064516129 Karen E. Drummond Karen Eich  Drummond
0.8181818181818182 Anita Sperling A. Sperling
0.7368421052631579 Janet Ahlberg AHLBERG
0.9166666666666666 Sarah Garland Sarah Galland
0.7333333333333333 Elizabeth Fletcher Betty Fletcher
0.8333333333333334 Richard N. Côté Richard N. Cote
0.8333333333333334 Jan Potocki Jan Graf Potocki
0.9032258064516129 Margaret Ruth Miles Margaret R. Miles
0.72 Christine (de Pisan) Christine
0.967741935483871 Kenneth E. Ethridge Kenneth Ethridge
0.896551724137931 Sir Douglas Mawson Douglas Mawson
0.9565217391304348 Julie Doucet Juliet Doucet
0.75 Jean-Paul Nozière Noziere J.-P.
0.9655172413793104 S. Scott Zimmerman Scott Zimmerman
0.9629629629629629

In [91]:
fifth_estimate = len(books[books.correct_author==""])

In [93]:
print("With the similarity threshold, we didn't find the same author for", fifth_estimate,"books out of ~269K. At this point, it is hard to further process.")

With the similarity threshold, we didn't find the same author for 21817 books out of ~269K. At this point, it is hard to further process.


## First alt author
We will use when running the viaf API

In [97]:
books["alt_first_author"] = books.alt_author.apply(lambda x: x.split("|")[0])

In [98]:
books.head()

,ISBN,title,author,year,publisher,alt_title,alt_author,correct_author,alt_first_author
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,Classical Mythology,Mark P. O. Morford|Robert J. Lenardon,Mark P. O. Morford,Mark P. O. Morford
1,0801319536,Classical Mythology,Mark P. O. Morford,1998,John Wiley &amp; Sons,Classical Mythology,Mark P. O. Morford|Robert J. Lenardon,Mark P. O. Morford,Mark P. O. Morford
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,Clara Callan,Richard Bruce Wright,Richard Bruce Wright,Richard Bruce Wright
3,0771597185,The teacher's daughter,Richard Bruce Wright,1982,Macmillan of Canada,The Teacher's Daughter,Richard Bruce Wright,Richard Bruce Wright,Richard Bruce Wright
4,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,Decision in Normandy,Carlo D'Este,Carlo D'Este,Carlo D'Este


In [104]:
books[books.correct_author!=""]

,ISBN,title,author,year,publisher,alt_title,alt_author,correct_author,alt_first_author
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,Classical Mythology,Mark P. O. Morford|Robert J. Lenardon,Mark P. O. Morford,Mark P. O. Morford
1,0801319536,Classical Mythology,Mark P. O. Morford,1998,John Wiley &amp; Sons,Classical Mythology,Mark P. O. Morford|Robert J. Lenardon,Mark P. O. Morford,Mark P. O. Morford
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,Clara Callan,Richard Bruce Wright,Richard Bruce Wright,Richard Bruce Wright
3,0771597185,The teacher's daughter,Richard Bruce Wright,1982,Macmillan of Canada,The Teacher's Daughter,Richard Bruce Wright,Richard Bruce Wright,Richard Bruce Wright
4,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,Decision in Normandy,Carlo D'Este,Carlo D'Este,Carlo D'Este
...,...,...,...,...,...,...,...,...,...
269269,0762412119,"Burpee Gardening Cyclopedia: A Concise, Up to ...",Allan Armitage,2002,Running Press Book Publishers,Burpee Gardening Cyclopedia,Maureen Heffernan|Holly H. Shimizu|Allan Armit...,Allan Armitage,Maureen Heffernan
269270,1582380805,Tropical Rainforests: 230 Species in Full Colo...,"Allen M., Ph.D. Young",2001,Golden Guides from St. Martin's Press,Tropical Rainforests,Allen M. Young,Allen M. Young,Allen M. Young
269271,1845170423,Cocktail Classics,David Biggs,2004,Connaught,Cocktail Classics,David Biggs,David Biggs,David Biggs
269272,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,From One to One Hundred,Teri Sloat,Teri Sloat,Teri Sloat


## Save

In [99]:
books.to_csv("data/items_books_for_viaf.csv")